# 二、复杂HTML解析

## 2.1 BeautifulSoup选择标签

### 2.1.1 findAll() 与 get_text()

注意，使用get_text()方法会清楚正在处理的HTML文档中的所有标签，然后返回一个只包含文字的Unicode字符串。

因此，我们通常在准备打印、存储和操作最终数据时，应该最后才使用get_text()方法。

一般情况下，你应该尽可能地保留HTML文档的标签结构。

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://pythonscraping.com/pages/warandpeace.html')
bs = BeautifulSoup(html.read(), 'html.parser')

nameList = bs.findAll('span', {'class': 'green'})
for name in nameList:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


### 2.1.2 find() 和 find_all()

两者的定义：

- find_all(tag, attributes, recursive, string, limit, keywords)
- find(tag, attributes, recursive, string, keywords)

可以看到，这两个方法的定义很类似。以下是参数介绍：

- tag：标签名称 或 标签列表，如'h1', ['h1', 'h2', 'h3']等。
- attributes：属性字典，封装一个标签的若干属性和对应的属性值，如{ 'class': { 'green', 'red' } }。
- recursive：递归参数，布尔值，表示是否需要递归查询，默认值为True，此时会递归查找标签；而为False时只查询文档的一级标签。通常在结构明确且要求抓取速度时手动设置为False。
- string：文本参数，表示用标签的文本内容去匹配，而不使用标签的属性，如'the prince'。
- limit：范围限制参数（仅用于find_all()方法）。find其实等价于limit=1时的find_all()。表达你需要找到网页中的前limit项结果。不过结果的顺序和你想要的顺序可能不一致，默认的结果顺序是按照网页上的顺序排序的。
- keyword：关键词参数，用于选择具有指定属性的标签，如id='title', class_='text'（class写成class_是为了避免python解释器将其视为class关键字）

In [4]:
h_tag = bs.find_all(['h1', 'h2', 'h3', 'h4'])
print(f"{h_tag=}")
attr_tag = bs.find_all('span', {'class': {'green', 'red'}})
print(f"{len(attr_tag)=}")
text_tag = bs.find_all(string='the prince')
print(f"{len(text_tag)=}")
title_tag = bs.find_all(id='title', class_='text')
# 等价写法：
title_tag2 = bs.find_all('', {'id': 'title', 'class': 'text'})
print(f"{title_tag=}")
print(f"{title_tag2=}")

h_tag=[<h1>War and Peace</h1>, <h2>Chapter 1</h2>]
len(attr_tag)=75
len(text_tag)=7
title_tag=[]
title_tag2=[]


### 2.1.3 其他的BeautifulSoup对象

BeautifulSoup库中的所有对象：

- BeautifulSoup对象
- Tag对象：表示标签的对象
- NavigableString对象（不常用）：用于表示标签里的文字，而不是标签本身（有些函数可以操作和生成NavigableString对象，而不是标签对象）
- Comment对象（不常用）：用于查找HTML文档的注释标签

### 2.1.4 导航树

通过导航树（navigating trees），我们可以通过标签在文档中的位置来查找标签。

例如：http://www.pythonscraping.com/pages/page3.html页面的标签树如下：

- HTML
  - body
    - div.wrapper
      - h1
      - div.content
      - table#giftList
        - tr
          - th
          - th
          - th
          - th
        - tr.gift#gift1
          - td
          - td
            - span.excitingNote
          - td
          - td
            -img
        - ...其他表格行（这里省略不写）
      - div.footer

1. 处理 子标签 和 后代标签

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

for child in bs.find('table', {'id': 'giftList'}).children:
    print(child)



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


2. 处理 兄弟标签

注意：由于页面的布局可能会随着时间不断发生改变，因此，我们最好使用属性来定位标签，而不是使用布局结构来定位标签。使用属性的爬虫往往更加稳定。

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

for sibling in bs.find('table', {'id': 'giftList'}).tr.next_siblings:
    print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

3. 处理 父标签

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')

print(bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())


$15.00



## 2.2 正则表达式

“如果你有一个问题打算用正则表达式来解决，那么就是两个问题了。”——计算机科学领域的一个笑话

正则表达式（简写为regex），主要用于复杂查找和过滤，相比于写一堆查找和过滤函数，还是正则表达式更简单。

非正则表达式同样存在，例如回文等，不过好在非正则表达式在网络爬虫领域的需求中很少见。

正则表达式：

- a：就是1个a，可以自行修改
- a*：a出现0次或多次
- a+：a出现1次或多次
- \[abc\]：abc中任选其一
- (aa)：编组之后得到的aa视为一个整体
- {m,n}：匹配前面的字符、子表达式、组出现最少m次，最多n次
- \[\^abc\]：abc之外的字符任选一个
- \|：分割的子表达式、符号任选其一，如b(a|b|c)c -> bac, bbc, bcc
- \.：匹配任意单个字符（包括符号、数字和空格等），如a.b -> aab, abb, acb, a1b, a b
- \^：表示一个字符串的开始位置的 字符或者子表达式，如^a -> a, ab, abc
- \：转义字符，将有特殊含义的字符穿换位字面形式，如\\.\\^\\\\ -> .^\
- \$：常用于正则表达式的末尾，表示从字符串的末尾开始匹配。如果不用它，每个正则表达式实际都带着".\*"模式（即后面可以接任意字符）。可以看成是^符号的反义词。如\[A-Z\]\*\[a-z\]\*\$ -> ABCabc, zzzyx, Bob
- ?!：“不包含”，通常放在字符或者正则表达式的前面，表示字符不能出现在目标字符串里。这个符号比较难用，。如果要在整个字符串中彻底排除某个字符，就加上^和$符号。如\^\(\(?!\[A-Z\]\)\.\)\*\$ -> no-caps-here, $ymb01sa3e f!ne

正则表达式练习——邮箱匹配：

\[A-Za-z0-9\\\.\\+-\]+@[A-Za-z0-9]+\\\.\(com\|cn\|org\|edu\|net\)

注意，并非所有的正则表达式都一样，但是大部分都相同。如果遇到不同版本的正则表达式，一定要阅读文档进行比对。比如Java中的正则表达式就和Python不一样。

## 2.3 正则表达式 + BeautifulSoup

正则表达式可以作为BeautifulSoup语句的任意一个参数，让你可以灵活地查找目标元素。

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://www.pythonscraping.com/pages/page3.html')
bs = BeautifulSoup(html, 'html.parser')
images = bs.find_all('img', {'src': re.compile('\.\.\/img\/gifts\/img.*\.jpg')})
for image in images:
    print(image['src'])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


## 2.4 Lambda表达式

Lambda表达式本质就是匿名函数，可以作为变量传入另一个函数。

BeautifulSoup允许我们把特定类型的函数作为参数传入find_all()函数。唯一的限制条件是：这些函数必须把一个标签对象作为参数并且返回布尔类型的结果。

BeautifulSoup用这个函数来评估它遇到的每个标签对象，最后把评估结果为True的标签保留，把其他标签剔除。

In [11]:
two_attrs = bs.find_all(lambda tag: len(tag.attrs) == 2)
print(len(two_attrs))

special = bs.find_all(lambda tag: tag.get_text() == "Or maybe he's only resting?")
print(special)

6
[<span class="excitingNote">Or maybe he's only resting?</span>]
